In [1]:
# Following https://link.springer.com/article/10.1007/s11142-023-09764-x
import re
import numpy as np
import pandas as pd
import duckdb as db
import datetime as dt
from tqdm.notebook import tqdm
from collections import Counter
tqdm.pandas()

In [2]:
detail = pd.read_pickle('./wrds_transcript_detail.pkl', compression='zip')
component = pd.read_pickle('./transcript_ea.pkl', compression='zip')
lm = pd.read_csv('./Loughran-McDonald_MasterDictionary_1993-2024.csv')

In [3]:
component = db.query(
    """
    SELECT
      transcriptid,
      transcriptcomponenttypeid,
      STRING_AGG(componenttext, ' ') AS componenttext
    FROM component
    WHERE transcriptcomponenttypeid IN (3, 4)
    GROUP BY transcriptid, transcriptcomponenttypeid
    """
).df()
component['componenttext'] = component['componenttext'].progress_apply(
    lambda x: re.sub(
        r'\s+', ' ',
        re.sub(
            r'[^\w\s]', ' ',
            x.encode('utf-8', 'ignore').decode('unicode_escape', errors='ignore')
        )
    ).lower()
)
component = db.query(
    """
    select distinct a.*, b.mostimportantdateutc, b.mostimportanttimeutc, b.companyid
    from component as a
    inner join detail as b
    on a.transcriptid = b.transcriptid
    /* consider all conference matched as earnings call */
    /* where b.keydeveventtypename = 'Earnings Calls' */
    """
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  0%|          | 0/595953 [00:00<?, ?it/s]

C:\Users\Zhizhe Liu\AppData\Local\Temp\ipykernel_30296\3129596414.py:17: DeprecationWarning: invalid escape sequence '\s'
  x.encode('utf-8', 'ignore').decode('unicode_escape', errors='ignore')
C:\Users\Zhizhe Liu\AppData\Local\Temp\ipykernel_30296\3129596414.py:17: DeprecationWarning: invalid escape sequence '\h'
  x.encode('utf-8', 'ignore').decode('unicode_escape', errors='ignore')
C:\Users\Zhizhe Liu\AppData\Local\Temp\ipykernel_30296\3129596414.py:17: DeprecationWarning: invalid escape sequence '\A'
  x.encode('utf-8', 'ignore').decode('unicode_escape', errors='ignore')
C:\Users\Zhizhe Liu\AppData\Local\Temp\ipykernel_30296\3129596414.py:17: DeprecationWarning: invalid escape sequence '\.'
  x.encode('utf-8', 'ignore').decode('unicode_escape', errors='ignore')
C:\Users\Zhizhe Liu\AppData\Local\Temp\ipykernel_30296\3129596414.py:17: DeprecationWarning: invalid escape sequence '\i'
  x.encode('utf-8', 'ignore').decode('unicode_escape', errors='ignore')
C:\Users\Zhizhe Liu\AppData\Lo

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
component = component.sort_values(['transcriptid', 'transcriptcomponenttypeid'])
component['mostimportantdatetimeutc'] = component['mostimportantdateutc'].astype(str) + ' ' + component['mostimportanttimeutc'].astype(str)
component['mostimportantdatetimeest'] = pd.to_datetime(component['mostimportantdatetimeutc']).dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
component['mostimportantdatetimeest'] = component['mostimportantdatetimeest'].dt.tz_localize(None)
component['mostimportantdateest'] = component['mostimportantdatetimeest'].dt.date
component = component.drop(columns=['mostimportantdatetimeutc', 'mostimportantdateutc', 'mostimportanttimeutc'])
component['year'] = component['mostimportantdatetimeest'].dt.year
component['words'] = component['componenttext'].str.split()

In [8]:
lm = lm.loc[(lm['Positive'] > 0) | (lm['Negative'] > 0)]
res = []
for _, row in tqdm(lm.iterrows(), total=len(lm)):
    if row['Positive'] == 0:
        yr_added = row['Negative']
        yr_range = range(yr_added, 2025)
        res.append(
            {
                'word': row['Word'].lower(),
                'year': yr_range,
                'negative': 1,
                'positive': 0
            }
        )
    elif row['Negative'] == 0:
        yr_added = row['Positive']
        yr_range = range(yr_added, 2025)
        res.append(
            {
                'word': row['Word'].lower(),
                'year': yr_range,
                'negative': 0,
                'positive': 1
            }
        )
res = pd.DataFrame(res)
res = res.explode('year')
res = res.loc[~((res['word'] == 'question') | (res['word'] == 'questions'))]
pos_year = res[res['positive'] == 1].groupby('year')['word'].apply(set).to_dict()
neg_year = res[res['negative'] == 1].groupby('year')['word'].apply(set).to_dict()

  0%|          | 0/2692 [00:00<?, ?it/s]

In [9]:
component['pos_count'] = 0
component['neg_count'] = 0

for idx, row in tqdm(component.iterrows(), total=len(component)):
    y = row['year']
    words = row['words']

    pos_words = pos_year.get(y, set())
    neg_words = neg_year.get(y, set())

    wc = Counter(words)

    pos_count = sum(wc[w] for w in wc if w in pos_words)
    neg_count = sum(wc[w] for w in wc if w in neg_words)

    component.at[idx, 'pos_count'] = pos_count
    component.at[idx, 'neg_count'] = neg_count

component = component.drop(columns=['words'])
component['sen_count'] = component['pos_count'] + component['neg_count']
component['transcriptcomponenttype'] = np.where(
    component['transcriptcomponenttypeid'] == 3,
    'analyst',
    'manager'
)
component = component.drop(columns=['transcriptcomponenttypeid', 'componenttext'])

  0%|          | 0/607954 [00:00<?, ?it/s]

In [16]:
tone = component.pivot(
    index=['transcriptid', 'mostimportantdateest', 'companyid', 'year'],
    columns='transcriptcomponenttype',
    values=['pos_count', 'neg_count', 'sen_count']
)
tone.columns = [f"{val}_{col}" for val, col in tone.columns]
tone = tone.reset_index()
tone.loc[tone['sen_count_manager'] != 0, 'tone_manager'] = (tone['pos_count_manager'] - tone['neg_count_manager']) / tone['sen_count_manager']
tone.loc[tone['sen_count_analyst'] != 0, 'tone_analyst'] = (tone['pos_count_analyst'] - tone['neg_count_analyst']) / tone['sen_count_analyst']

In [9]:
tone[['transcriptid', 'tone_analyst', 'tone_manager']].to_pickle('./tone.pkl', compression='zip')